In [3]:
import torch
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import tqdm

In [4]:
TEST_IMAGE_DIRS = "./data/test_images/"
BATCH_SIZE = 32
df_test = pd.read_csv("./data/sample_submission.csv")

In [5]:
class CassavaTestDataset(Dataset):
    def __init__(self, df, img_name_col, img_root_folder, transform=None):
        self.df = df
        self.img_name_col = img_name_col
        self.img_root_folder = img_root_folder
        self.transform = transform        

    def __getitem__(self, index):
        img_path = self.img_root_folder + "/" + self.df.loc[index, self.img_name_col]
        img = Image.open(img_path)        
        if self.transform is not None:
            img = self.transform(img)
        return img

    def __len__(self):
        return len(self.df)

In [6]:
test_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),        
        transforms.Normalize([0.4303, 0.4967, 0.3135], [0.2203, 0.2232, 0.2114])
])

In [10]:
ds_test = CassavaTestDataset(
        df_test, 
        img_name_col="image_id",        
        img_root_folder="./data/test_images", 
        transform=test_transform
        )        

dl_test = DataLoader(ds_test, batch_size=BATCH_SIZE)    

In [8]:
import torchvision.models as models
import torch.nn as nn
import torchmetrics

class ResnetTL_LitModel(pl.LightningModule):
    def __init__(self, num_classes, fine_tune=True, drop_out=0.25, lr=0.00012):
        super().__init__()
        pt_model = models.resnet50(pretrained=True)
        self.num_classes = num_classes
        self.lr = lr
        self.fine_tune = fine_tune
        self.drop_out = drop_out        
        self.train_loss = nn.CrossEntropyLoss()
        self.val_loss = nn.CrossEntropyLoss()
        self.train_accuracy = torchmetrics.Accuracy() 
        self.val_accuracy = torchmetrics.Accuracy()
        # We cut the head from the pretrained model and use the remaining model as a feature extractor
        pt_layers = list(pt_model.children())[:-1]
        self.backbone = nn.Sequential(*pt_layers)
        # Create a new model head
        # For resnet50 the num of input features to final FC layer is 2048
        self.new_head = self.create_model_head(pt_model.fc.in_features)        

    def create_model_head(self, in_features):         
        # Create a new model head 
        # The weights of the newly created layers are initialized by pytorch automatically
        # See : https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/linear.py#L81
        new_head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(p=self.drop_out),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=self.drop_out),
            nn.Linear(256, self.num_classes)
        )                
        return new_head

    def forward(self, x):
        features = self.backbone(x)
        features = torch.flatten(features, 1)
        x = self.new_head(features)
        return x

    def configure_optimizers(self):
        model_optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(model_optimizer, "min")        
        return {
            "optimizer": model_optimizer, 
            "lr_scheduler": {
                "scheduler": lr_scheduler,
                "monitor": "val_loss",
                "frequency": 1
            }
        }

    def training_step(self, batch, batch_idx):
        X, y = batch
        y_pred = self(X)        
        loss = self.train_loss(y_pred, y)
        acc = self.train_accuracy(y_pred, y)
        self.log("train_loss", loss, on_epoch=True, logger=True, prog_bar=True)
        self.log("train_accuracy", acc, on_epoch=True, logger=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y = batch
        y_pred = self(X)
        loss = self.val_loss(y_pred, y)
        acc = self.val_accuracy(y_pred, y)
        self.log("val_loss", loss, on_epoch=True, logger=True, prog_bar=True)
        self.log("val_accuracy", acc, on_epoch=True, logger=True, prog_bar=True)
        return loss

In [11]:
model = ResnetTL_LitModel.load_from_checkpoint(
    "./model/cassava_best_model.ckpt",     
    num_classes=5
    )
model.to("cuda")
model.eval()
predictions = []
with torch.no_grad():
    counter=0
    for imgs in tqdm.tqdm(dl_test):                
        predicted_cuda_labels = torch.argmax(model(imgs.to("cuda")), dim=1)
        predicted_labels = predicted_cuda_labels.cpu().detach()
        predictions.extend(predicted_labels.numpy().tolist())
df_test["label"] = predictions
df_test[["image_id","label"]].to_csv("submission.csv",index=False)
df_test.head()

100%|██████████| 1/1 [00:07<00:00,  7.51s/it]


,image_id,label
0,2216849948.jpg,4
